# Genome Assembly

![Newspaper Analogy from CSE 181 online notes](imgs/newspaper_blowup.png)

## Overview:
Genome Sequencing is difficult. We cannot simply read from one end of a chromosome to another (at least not yet). **So instead we must cut each chromosome into many small pieces and sequence the individual parts.** Reconstructing the original chromosomes using the small pieces is called Genome Assembly.

Just like we can find overlapping parts in the newspaper scraps, we assemble the original genome by finding overlapping sequences in each read. Then we just read the entire sequence from left to right.

![Sequencing overview from CSE 181 online material](imgs/sequencing_overview.png)

Although this process seems simple, there are a few wrinkles which complicate the process:

1. DNA is double stranded so the overlapping sequences may be on opposite strands-we have to deal with reverse compliment sequences.
2. Sequencing is not perfect and errors are still common. 
3. Some regions of the genome may not be covered by any reads.

Though for our excercises we're mostly going to ignore these problems. We can pretend that there are no missing segments or errors in our sequences and that all sequences are on the same strand.

## Python functions and dictionaries

Given that we can store strings in Python, we can actually store different data types that are related to one another with a dictionary. 

For each entry in a dictionary we have 2 pieces: a key and a value. The key is an identity or the name of a location. The value is what is stored at that location and is returned when using the key. In a dictionary the key is a word and the value is the definition, you get by navigating to the location of the key in the book.

In other programming languages dictionaries are called *hashmaps*. They are very fast but can occasionally take a lot of space in memory.

Here is a [quick tutorial on dictionaries](https://www.w3schools.com/python/python_dictionaries.asp) and below is a quick summary.

---

What if we wanted to print the name and first 3 letters of each students major in the example below. 


In [ ]:
# Example: Let's say you want to store a list of students & their majors
students = {
    "Joe": "Biology",
    "Anna": "Psychology",
    "Bob": "History"
}

# How would I print the name and major of just one student:
name_and_3let_major_string = ""
print(name_and_3let_major_string)

Instead of repeating it three times (once for each student), what if we wrote a function and a for loop. We'll start with the function:

In [ ]:
def name_and_major(dictionary, key):
    """
    Return the name and 3 letter major string from the dictionary for a given key
    """
    name_and_3let_major_string = ""
    # TODO
    return name_and_3let_major_string

# Test case for function
print(name_and_major(students, "Joe"))

Now let's try using the function with a for loop.

Hint: The test case above shows how to call your function.

In [ ]:
for key in students.keys:
    # TODO
    pass

## Kmer Introduction
Kmers are short DNA sequences of a given length that we can use to construct a DNA sequence.

For example given the sequence `CAATCCAAC`. The first 5-mer present would be `CAATC`, the second `AATCC` and so on. These kmers let us represent a longer sequence as the composition of many smaller pieces of length k. 

If we can imagine the cut-up pieces of DNA during sequencing as kmers in the genome we might be able to see how algorithms with kmers could be useful for genome assembly. In fact, kmers are fundamental to many bioinformatic algorithms. 

---

Let's practice generating them.

hint: a `for` loop and string slicing should be enough to solve this problem.

In [ ]:
def kmer_composition(text: str, k: int) -> list[str]:
    """
    This function should return a list containing all kmers of length k from text
    """
    kmers = []
    # TODO

    return kmers

In [ ]:
# Check you solution here
k = 5
text = "CAATCCAAC"
correct_output = ["CAATC", "AATCC", "ATCCA", "TCCAA", "CCAAC"]
computed_output = kmer_composition(text, k)

if len(correct_output) is not len(computed_output): raise Exception("Wrong number of kmers")
for kmer in correct_output:
    if kmer not in computed_output:
        raise Exception(f"Missing kmer: {kmer}")
print("Success!")

My solution:

In [ ]:
# TODO: My solution
def kmer_composition(text: str, k: int) -> list[str]:
    """
    This function should return a list containing all kmers of length k from text
    """
    kmers = []
    for i in range(len(text)-k+1):
        kmers.append(text[i:i+k])
    return kmers

In [ ]:
# What should the output of the function will be?
kmer_composition("AATCGATC", 3)

## Does aligning kmers always work?

Given our set of kmers, one way to reconstruct our original sequence is to notice their overlaps. In the previous example our contigs overlap as shown:

``` python
CAATC
 AATCC
  ATCCA
   TCCAA
    CCAAC
```
Each column corresponds to a single nucleotide and we can reconstruct the sequence by reading from left to right.

``` python
CAATCCAAC
```


More concretely, each k-mer is made up of a **prefix** and a **suffix** each of length k-1. Then overlap between k-mers A and B is equivalent to saying the suffix of k-mer A is identical to the prefix of k-mer B.

For example let kmer_A and kmer_B be defined below:
``` python
kmer_A = "CAATC"
kmer_B = "AATCC"
```

What is the prefix and suffix of kmer_A? kmer_B?

In [4]:
# TODO: insert answers here
kmer_A_prefix = ""
kmer_A_suffix = ""

kmer_B_prefix = ""
kmer_B_suffix = ""

Which of the following statements will print True?

In [ ]:
print("Case A:", kmer_A_prefix == kmer_B_prefix)
print("Case B:", kmer_A_prefix == kmer_B_suffix)
print("Case C:", kmer_A_suffix == kmer_B_prefix)
print("Case D:", kmer_A_suffix == kmer_B_prefix)

Now try assembly in practice to get an intuition for our goal. 

Consider this set of 3-mers. Take a few minutes and try to reconstruct the original sequence. Type your answer below.
``` python
AAT AAT ATG ATT GAA TAA TGA TTA
```

Try adding in the overlapping segments here like we saw before:
``` python
# TODO: 
SEQA
 SEQB
  SEQC
   ...

# Solution:
```

In [2]:
#TODO hide solution
original_seq = "TAATGAATTA"

You may notice that connecting the overlapping pieces doesn't always work. Consider this example:
``` python
TAA
 AAT
  ATT
   TTA
```
After assembling 4 kmers we find that there is no kmer that we can add after TTA. This is because one k-mer, AAT, is repeated twice. We have to guess the correct kmer to follow afterward in either ATG or ATT. 

Recall our genomes are made up of millions of repeated segments. To test every possible combination is an extremely slow task, even with modern computing power.

Instead we must come up with a more clever approach to solve this problem.

## Graph introduction, Overlap Graphs and de Bruijn graphs

TODO

1. Introduction to directed graphs.
    1. A graph is a set of nodes and edges.
        1. Also introduce the in-degree and out-degree of a node.
    2. We represent a di-graph in python as a dictionary of nodes mapping to other nodes it connects to. (practice this on trivial case probably)
2. Then introduce overlap graphs. 
    1. Practice reading the original sequence back from an overlap graph, and how to construct one using a set of kmers. Especially when the nodes are not in the correct order.
    2. The original sequence in an overlap graph is a path which visits all nodes.
    3. Finding this path (eulerian path) is slow so instead we use a slightly different graph
3. Then introduce de Bruijn graph == overlap graph but compress identical nodes into one
    1. Then the original sequence in this graph is a path that visits every edge. (Eulerian path)
    2. Practice reading it out.

Even though we introduced de Bruijn graphs from overlap graphs. We can construct them directly from a set of kmers. We don't need the original sequence at all.

`For each kmer, add an edge from its prefix to its suffix to our graph.`

For example:
``` python
de_bruijn_graph = defaultdict(list)
kmer = "ACGT"
de_bruijn_graph["ACG"].append("CGT")
```

Try implementing a function to build a de Brujin graph from a list of kmers. I've started the function below for you:

In [1]:
from collections import defaultdict

In [ ]:
def build_de_bruijn(kmers:list[str]) ->  Dict[str, List[str]]:
    de_bruijn_graph = defaultdict(list)
    for kmer in kmers:
        #TODO Your solution here
    return de_bruijn_graph

In [6]:
# TODO hide test function for de_bruijn graph
"""
Sample input k and string
3
ACGTGTATA
"""
"""
Sample output edge list
AC: CG
AT: TA
CG: GT
GT: TA TG
TA: AT
TG: GT
"""

'\nSample output edge list\nAC: CG\nAT: TA\nCG: GT\nGT: TA TG\nTA: AT\nTG: GT\n'

## Finding genomes in de Bruijn graphs

TODO: 
1. Introduce Eulerian Cycle
    1. Recall we want to find an path that visits every edge (Eulerian Path)
    2. One of the challenges is that we don't even know where to start.
    3. Idea: if we connect the first and last node by an edge. We can start looking from anywhere. --> Eulerian Cycle
2. We can identify the first and last node because they are the only nodes with an odd number of edges. 
    1. Ant traveling along a graph analogy. If there are an odd number of edges to a node we must either start or finish there. Otherwise we only visit and then leave using the corresponding even node
    2. We can have them implement this part in an example
3. Then we can use efficient algorithms for finding an eulerian cycle
    1. Hierholzer's Algorithm, may have time to explain with some figures but definitely we won't be implementing it.


## Addendum: assembly in practice

Assembly in reality is much more difficult than we have illustrated here because we made several simplifying assumptions.

Firstly, to reconstruct a genome by traversing a de Bruijn graph we assumed that all kmers of our original sequence are present. But in reality our reads are not evenly distributed. In the figure below we can see that even though the sequenced 10-mers cover the entire sequence, they are too sparse to use the method we've described above. To accomodate for this problem we can break the 10-mers into smaller k-mers. These smaller 5-mers cover every possible 5-mer in our sequence so our assembly method works.

However, this is not a catch-all. When lowering our k, we are more likely to see repetitive sequences and this can make multiple correct solutions possible. So we want to maintain as high k as possible while covering all k-mers in the dataset.

![read breaking from CSE 181 lecture notes](imgs/read_breaking.png)

Secondly, we assumed that there were no errors in any of our reads. An error in a read produces a *bubble* in our overlap graph as shown below. Modern assemblers will try to detect these bubbles in de Bruijn graphs collapse them to the correct path based on the number of reads that support each variation. We can predict the size of bubbles based on which k we choose to better detect erroneous reads.

But it's not a perfect process and errors do occasionally make thier way into assemblies.

![bubble](imgs/bubble.png)

We also assumed that there are no gaps in our reads coverage of the genome. This is very rarely the case. In this scenario we cannot generate a sequence that is not present in our data. So in practice, we must settle for the longest contigous sequences possible, these are often called **contigs**. Fortunately these can also be found efficiently in the de Bruijn graph as *maximal non-branching paths*. Each non-branching path corresponds to a sequence that we are 100% sure exists as a real sequence in the genome because it has no ambiguity on which path it follows.

Each contig must exist as a real contiguous sequence on one chromosome, but the contigs could exist very far apart from each other or on different chromosomes entirely. We can scaffold these contigs together into an assembly with gaps using outside information or different sequencing technologies (Hi C, Long Read, paired-end).

The complete human genome published in 2000 was such an assembly with gaps comprising ~8% of all bases. The real gapless human genome was only completed in 2022.

In practice sequencing is often done with paired-end reads. This allows us to know that two sequences are a fixed distance apart from each other. In many cases, we can use this information to eliminate incorrect, but possible assemblies from the real contig that exists in the organism.

We assumed that all reads were on the same strand of our double stranded DNA. In reality, we sequence from both strands of DNA and have no way to determine which is which. Our solution is to add the reverse-compliment of every read into our dataset, effectively doubling the number of reads. This complicates assembly but allows to include all information from sequencing.

This extra complexity in our de Bruijn graph is not random. Certain characteristic features appear such as palindromes and we can simplify our graph without losing information to better handle the influx of reads. This combined with paired-end reads still allows us to find contigs effectively.